<a href="https://colab.research.google.com/github/dominicwhat/Data-AI---RecSys/blob/main/sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2021-12-26 15:55:07--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.7’

All_Beauty.csv.7    100%[===================>]  14.78M  24.3MB/s    in 0.6s    

2021-12-26 15:55:08 (24.3 MB/s) - ‘All_Beauty.csv.7’ saved [15499476/15499476]

--2021-12-26 15:55:08--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.7’

meta_All_Beauty.jso 100%[===================>]   9.85M  18.7MB/s    in 0.5s   

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [5]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [6]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## 資料切分

1.將2018年8月前(含8月)的資料當training資料，2018年9月的資料當成testing資料。

In [7]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

2.我們將資料依照商品分組，並計算用戶對商品評價的次數，並以此次數重新做商品排序，重新製作dataset

In [8]:
asin_group_all = ratings_trainings.groupby(by=['asin']).agg(list).reset_index()
asin_group_all['interaction_num'] = [len(i) for i in asin_group_all['overall']]
asin_group_all = asin_group_all.sort_values(by='interaction_num', ascending=False,ignore_index = True)
asin_group_all.head()

,asin,reviewerID,overall,unixReviewTime,DATE,interaction_num
0,B000FOI48G,"[A213675GUCM3PV, AISWP2UHZT1S8, A4X295XP5LNY5,...","[1.0, 1.0, 3.0, 5.0, 5.0, 4.0, 1.0, 5.0, 5.0, ...","[1150848000, 1150848000, 1147305600, 114670080...","[2006-06-21 00:00:00, 2006-06-21 00:00:00, 200...",8672
1,B000GLRREU,"[A2XAHEQG61OQN1, A2CBEG67C03N66, ACA9BA6Z21MLK...","[3.0, 5.0, 5.0, 5.0, 3.0, 2.0, 5.0, 4.0, 5.0, ...","[1163376000, 1163116800, 1163030400, 116303040...","[2006-11-13 00:00:00, 2006-11-10 00:00:00, 200...",8341
2,1620213982,"[A1DN3KBTCYBH6K, A1TA38LWZO0NZS, A2OZBC5SG575G...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[1337126400, 1330214400, 1326672000, 132209280...","[2012-05-16 00:00:00, 2012-02-26 00:00:00, 201...",4792
3,B001QY8QXM,"[A2XWDKDS11MRNA, A3H6V59LAH6KQL, A176TSE9ZRY4Y...","[5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[1279065600, 1278374400, 1258416000, 152815680...","[2010-07-14 00:00:00, 2010-07-06 00:00:00, 200...",4544
4,B01DKQAXC0,"[A2PSPEPE6KXP19, A5KF1QROM0DGR, A3T8QVVGBTV7SM...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, ...","[1442880000, 1442880000, 1442880000, 144279360...","[2015-09-22 00:00:00, 2015-09-22 00:00:00, 201...",4197


3.因為資料為時序型資料，將所有的training資料不分時間都放入模型進行訓練，效果未必比較好，因此我們考慮testing資料的月份，取出training資料同年前個月分的資料進行訓練。

In [9]:
ratings_trainings_aug =  ratings_trainings[
    (ratings_trainings['DATE'] >= '2018-08-01') & 
    (ratings_trainings['DATE'] <= '2018-08-31')
]
asin_group_aug = ratings_trainings_aug.groupby(by=['asin']).agg(list).reset_index()
asin_group_aug['interaction_num'] = [len(i) for i in asin_group_aug['overall']]
asin_group_aug = asin_group_aug.sort_values(by='interaction_num', ascending=False,ignore_index = True)
asin_group_aug.head()

,asin,reviewerID,overall,unixReviewTime,DATE,interaction_num
0,B01DKQAXC0,"[A2IW9URACR5LUD, A3G0PHYROM1VGD, A33JCUAU4IM01...","[4.0, 5.0, 3.0, 5.0, 1.0, 5.0, 5.0, 5.0, 5.0, ...","[1535673600, 1535673600, 1535673600, 153567360...","[2018-08-31 00:00:00, 2018-08-31 00:00:00, 201...",75
1,B00W259T7G,"[A27RB5EZMQ89GH, A9M07B0RC4SKM, A2HJCM2R7DSN8B...","[5.0, 5.0, 2.0, 5.0, 5.0, 4.0, 5.0, 5.0, 3.0, ...","[1534636800, 1534550400, 1534550400, 153455040...","[2018-08-19 00:00:00, 2018-08-18 00:00:00, 201...",53
2,B013XKHA4M,"[A2EEDBOGC7O1XA, A27FC5L33T8MCA, A1XKVLBN1Q61Q...","[3.0, 5.0, 4.0, 5.0, 5.0, 5.0, 1.0, 5.0, 5.0, ...","[1534550400, 1534550400, 1534464000, 153437760...","[2018-08-18 00:00:00, 2018-08-18 00:00:00, 201...",29
3,B012Z7IHHI,"[A2EDK0TRPF5FH7, ABQX6FMYK9AJ8, A8EVHOIPGE3HT,...","[5.0, 2.0, 5.0, 1.0, 5.0, 5.0, 3.0, 5.0, 1.0, ...","[1534291200, 1534204800, 1534204800, 153403200...","[2018-08-15 00:00:00, 2018-08-14 00:00:00, 201...",26
4,B01DLR9IDI,"[AUX122XW8ONG6, AUX122XW8ONG6, AUX122XW8ONG6, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[1534982400, 1534982400, 1534982400, 153498240...","[2018-08-23 00:00:00, 2018-08-23 00:00:00, 201...",26


4.因為資料為時序型資料，將所有的training資料不分時間都放入模型進行訓練，效果未必比較好，因此我們考慮testing資料的月份，取出training資料前一年同月分的資料進行訓練。

In [10]:
ratings_trainings_last_sep =  ratings_trainings[
    (ratings_trainings['DATE'] >= '2017-09-01') & 
    (ratings_trainings['DATE'] <= '2017-09-30')
]
asin_group_last_sep = ratings_trainings_last_sep.groupby(by=['asin']).agg(list).reset_index()
asin_group_last_sep['interaction_num'] = [len(i) for i in asin_group_last_sep['overall']]
asin_group_last_sep = asin_group_last_sep.sort_values(by='interaction_num', ascending=False,ignore_index = True)
asin_group_last_sep.head()

,asin,reviewerID,overall,unixReviewTime,DATE,interaction_num
0,B0010ZBORW,"[A1BTG38XSGYLW3, A3S3R88HA0HZG3, A3SLC8F6VIWXI...","[5.0, 5.0, 5.0, 4.0, 4.0, 5.0, 3.0, 5.0, 4.0, ...","[1506729600, 1506729600, 1506643200, 150664320...","[2017-09-30 00:00:00, 2017-09-30 00:00:00, 201...",400
1,B00W259T7G,"[A1RF9YK4BK5TRH, A2CNGARED6OWWU, A30Z0GG96D11E...","[3.0, 4.0, 3.0, 4.0, 5.0, 4.0, 5.0, 5.0, 5.0, ...","[1506729600, 1506729600, 1506729600, 150672960...","[2017-09-30 00:00:00, 2017-09-30 00:00:00, 201...",387
2,B01DKQAXC0,"[A1U73UEOOKPCAG, A3MTSOTN6KI5YQ, A3LHIPR8FUS9B...","[1.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, ...","[1506729600, 1506729600, 1506556800, 150655680...","[2017-09-30 00:00:00, 2017-09-30 00:00:00, 201...",124
3,B000050FDY,"[AEULHIYYGAGE2, A4QU03PP6IKZ, A3O00BUEBIFVAR, ...","[5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[1506729600, 1506729600, 1506729600, 150672960...","[2017-09-30 00:00:00, 2017-09-30 00:00:00, 201...",103
4,B001LNODUS,"[A16WPA6JV83YXT, A10ZBR6O8S8OCY, A9P07NJ7UV0M,...","[2.0, 4.0, 2.0, 5.0, 3.0, 3.0, 4.0, 5.0, 5.0, ...","[1506729600, 1506643200, 1506556800, 150647040...","[2017-09-30 00:00:00, 2017-09-29 00:00:00, 201...",87


## 產生推薦

In [11]:
def recommender_random(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    random-based
    '''
    ratings_trainings = training_data
    recommendations = {user: ratings_trainings['asin'].sample(n=k).tolist() for user in users}
    return recommendations
ratings_by_user = recommender_random(ratings_trainings, users)

In [12]:
def recommender_asin_top(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    ratings_trainings = training_data
    recommendations = {user: ratings_trainings['asin'][0:k].tolist() for user in users}
    return recommendations
ratings_by_user_asin_all = recommender_asin_top(asin_group_all, users)
ratings_by_user_asin_aug = recommender_asin_top(asin_group_aug, users)
ratings_by_user_asin_last_sep = recommender_asin_top(asin_group_last_sep, users)


## 結果評估

由下述結果可看出，隨機推薦的表現最差，再來次好的推薦方法是不考慮時間的影響，直接找原始training資料前K個最多人評價的商品，再來才是推薦testing資料前月份前K個最多人評價的商品，最後最好的才是推薦testing資料前一年同月份前K個最多人評價的商品。

In [13]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score
print("Random:",evaluate(ratings_testings_by_user, ratings_by_user))
print("Asin_top_K_all:",evaluate(ratings_testings_by_user, ratings_by_user_asin_all))
print("Asin_top_K_aug:",evaluate(ratings_testings_by_user, ratings_by_user_asin_aug))
print("Asin_top_K_last_sep:",evaluate(ratings_testings_by_user, ratings_by_user_asin_last_sep))

Random: 0.013559322033898305
Asin_top_K_all: 0.08305084745762711
Asin_top_K_aug: 0.15423728813559323
Asin_top_K_last_sep: 0.1
